In [1]:
import polars as pl

import nwec.utility_reporting.arrearages
import nwec.utils.excel
from nwec.constants import DATA

YEAR = 2024
NUM_MONTHS = 12
COLS_PER_MONTH = 4
SHEET_SEARCH_STRING = "past due balances"
ARREARAGE_SEARCH_STRING = "past-due balances by customer class"
KLI_SEARCH_STRING = "past-due balances for known low-income household"
spreadsheet = DATA / f"utility_reporting/avista/avista_{YEAR}.xlsx"

In [2]:
sheet_index = nwec.utils.excel.get_sheet_index_from_name(spreadsheet, SHEET_SEARCH_STRING)
df = pl.read_excel(spreadsheet, sheet_id=sheet_index, has_header=False)
arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(
    df, NUM_MONTHS, COLS_PER_MONTH, ARREARAGE_SEARCH_STRING
)
kli_arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(df, NUM_MONTHS, COLS_PER_MONTH, KLI_SEARCH_STRING)

# Arrearages

In [3]:
date_to_zip_offset = 1
source_date_format = "%Y-%m-%d %H:%M:%S"

arrearages = nwec.utility_reporting.arrearages.rename_arrearage_columns(
    arrearages, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [4]:
arrearages.sum()

Zip Code,January 2024 30 Days,January 2024 60 Days,January 2024 90 Days +,January 2024 Total Arrearages,February 2024 30 Days,February 2024 60 Days,February 2024 90 Days +,February 2024 Total Arrearages,March 2024 30 Days,March 2024 60 Days,March 2024 90 Days +,March 2024 Total Arrearages,April 2024 30 Days,April 2024 60 Days,April 2024 90 Days +,April 2024 Total Arrearages,May 2024 30 Days,May 2024 60 Days,May 2024 90 Days +,May 2024 Total Arrearages,June 2024 30 Days,June 2024 60 Days,June 2024 90 Days +,June 2024 Total Arrearages,July 2024 30 Days,July 2024 60 Days,July 2024 90 Days +,July 2024 Total Arrearages,August 2024 30 Days,August 2024 60 Days,August 2024 90 Days +,August 2024 Total Arrearages,September 2024 30 Days,September 2024 60 Days,September 2024 90 Days +,September 2024 Total Arrearages,October 2024 30 Days,October 2024 60 Days,October 2024 90 Days +,October 2024 Total Arrearages,November 2024 30 Days,November 2024 60 Days,November 2024 90 Days +,November 2024 Total Arrearages,December 2024 30 Days,December 2024 60 Days,December 2024 90 Days +,December 2024 Total Arrearages
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,1.9802e6,1.2420e6,2.8916e6,6.1138e6,2.5340e6,1.6214e6,2.9906e6,7.1460e6,3.3460e6,2.3195e6,3.3744e6,9.0400e6,2.4698e6,2046855.3,3.0270e6,7.5437e6,2.2034e6,1.5750e6,2.7109e6,6.4892e6,1.8379e6,1.6149e6,2.5123e6,5.9651e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# KLI Arrearages

In [5]:
date_to_zip_offset = 1
source_date_format = "%Y-%m-%d %H:%M:%S"

kli_arrearages = nwec.utility_reporting.arrearages.rename_arrearage_columns(
    kli_arrearages, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [6]:
kli_arrearages.sum()

Zip Code,January 2024 30 Days,January 2024 60 Days,January 2024 90 Days +,January 2024 Total Arrearages,February 2024 30 Days,February 2024 60 Days,February 2024 90 Days +,February 2024 Total Arrearages,March 2024 30 Days,March 2024 60 Days,March 2024 90 Days +,March 2024 Total Arrearages,April 2024 30 Days,April 2024 60 Days,April 2024 90 Days +,April 2024 Total Arrearages,May 2024 30 Days,May 2024 60 Days,May 2024 90 Days +,May 2024 Total Arrearages,June 2024 30 Days,June 2024 60 Days,June 2024 90 Days +,June 2024 Total Arrearages,July 2024 30 Days,July 2024 60 Days,July 2024 90 Days +,July 2024 Total Arrearages,August 2024 30 Days,August 2024 60 Days,August 2024 90 Days +,August 2024 Total Arrearages,September 2024 30 Days,September 2024 60 Days,September 2024 90 Days +,September 2024 Total Arrearages,October 2024 30 Days,October 2024 60 Days,October 2024 90 Days +,October 2024 Total Arrearages,November 2024 30 Days,November 2024 60 Days,November 2024 90 Days +,November 2024 Total Arrearages,December 2024 30 Days,December 2024 60 Days,December 2024 90 Days +,December 2024 Total Arrearages
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,359026.37,385153.04,1.4268e6,2.1710e6,418069.06,500612.85,1.4692e6,2.3878e6,423795.41,480086.37,1.6715e6,2.5754e6,294259.5,428951.27,1.3864e6,2.1096e6,247913.34,269823.11,1.2266e6,1.7443e6,181198.94,252652.83,1.0977e6,1.5316e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
